In [1]:
import json
import librosa 
import numpy as np
import os
from tqdm.notebook import tqdm
import warnings
from audio2numpy import open_audio

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

import pickle

warnings.filterwarnings("ignore")


In [2]:
import sys
sys.path.append(r'../../..')
import VAT

In [3]:
# Set dataset path
datasetPath = r".."
rawDataPath = os.path.join(datasetPath, "raw")
outPath = "raw"

In [4]:
with open(os.path.join(datasetPath, "verified_sr_8000.pkl"), "rb") as f:
    data = pickle.load(f)

In [5]:
sampleRate = 8000

In [6]:
dataVAT = {
    "filename": [],
    "signal": [],
    "label": []
}

for filename, signal, label in tqdm(zip(data["filename"], data["signal"], data["covid"]), total = len(data["filename"])):
    pcm16 = VAT.float_to_pcm16(signal)
    frame_duration = 0.01
    vad_mode = 3
    vad_res = VAT.detectVoiceActivity(pcm16, sampleRate, frame_duration, vad_mode)
    sample, start = VAT.sampleVoiceActivity(signal, vad_res, frame_duration, sampleRate)

    if sample is None:
        continue

    dataVAT["filename"].append(filename)
    dataVAT["signal"].append(sample)
    dataVAT["label"].append(label)

  0%|          | 0/819 [00:00<?, ?it/s]

ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No active voice detected.
ERROR: No 

In [7]:
print("covid: 0, {}".format(np.sum(np.asarray(dataVAT["label"]) == 0)))
print("covid: 1, {}".format(np.sum(np.asarray(dataVAT["label"]) == 1)))

covid: 0, 364
covid: 1, 243


In [10]:
signals = np.asarray(dataVAT["signal"])

In [13]:
with open("vat_8000.pkl", "wb") as f:
    pickle.dump(dataVAT, f)